In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json
Saving resnet18-f37072fd.pth to resnet18-f37072fd.pth


In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset
!unzip new-plant-diseases-dataset.zip -d /content/

In [ ]:
!ls -l /content
import os
#loads resnet network
resnet = "/content/resnet18-f37072fd.pth"
print(os.path.exists(resnet))

total 2875556
-rw-r--r-- 1 root root         64 Sep  9 09:00  kaggle.json
drwxr-xr-x 3 root root       4096 Sep  9 09:02 'new plant diseases dataset(augmented)'
drwxr-xr-x 3 root root       4096 Sep  9 09:00 'New Plant Diseases Dataset(Augmented)'
-rw-r--r-- 1 root root 2897709187 Oct 12  2019  new-plant-diseases-dataset.zip
-rw-r--r-- 1 root root   46830571 Sep  9 09:00  resnet18-f37072fd.pth
drwxr-xr-x 1 root root       4096 Sep  5 13:36  sample_data
drwxr-xr-x 3 root root       4096 Sep  9 09:04  test
True


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np # linear algebra
import pandas as pd

# Paths
baseDir = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
trainDir = f"{baseDir}/train"
valDir = f"{baseDir}/valid"
#testDir  = f"{baseDir}/test/test"
# ignoring test for now!

# get everything to be uniform so we can train (224,224) for ResNet
# converts images to python Tensors (i.e. vectors)
# must match ImageNet as we are using ResNET, so we normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# put into datasets
trainDS = datasets.ImageFolder(root=trainDir, transform=transform)
valDS   = datasets.ImageFolder(root=valDir, transform=transform)
#testDS  = datasets.ImageFolder(root=testDir, transform=transform)

#js to check if loaded in our data correctly and with correct path
print("Training classes:", os.listdir(trainDir)[:10])
print("Validator classes:", os.listdir(valDir)[:10])
#print("Test classes:", os.listdir(test_dir)[:5])
print("# of training images:", len(trainDS))

# feeding data by batch 32 into model + randomizes to model for better training
trainLoader = DataLoader(trainDS, batch_size=32, shuffle=True)
valLoader   = DataLoader(valDS, batch_size=32, shuffle=False)
#testLoader  = DataLoader(testDS, batch_size=32, shuffle=False)


Training classes: ['Tomato___Early_blight', 'Tomato___Target_Spot', 'Tomato___Late_blight', 'Grape___Black_rot', 'Apple___healthy', 'Potato___Early_blight', 'Grape___healthy', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Strawberry___healthy']
Validator classes: ['Tomato___Early_blight', 'Tomato___Target_Spot', 'Tomato___Late_blight', 'Grape___Black_rot', 'Apple___healthy', 'Potato___Early_blight', 'Grape___healthy', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Strawberry___healthy']
# of training images: 70295


In [ ]:
import torchvision.models as models
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import torch

# we are using pre-trained resnet bc our dataset is wayyy too large to train every layer
# rather, we will just train the final layer, which classifies plants
# not something that needs to figure out how to read images as well
model = resnet18(weights=None)
pretrained_dict = torch.load("/content/resnet18-f37072fd.pth")
model_dict = model.state_dict()
# filter out final layer which is fc
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and "fc" not in k}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

#iterate through each neuron and doesn't allow for computing gradients (no backpropagation = no changing this layer)
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

#access last layer (model.fc) to change output to plant classification
num_classes = len(trainDS.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:
#using Colab's GPU space
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Using device:", device)

Using device: cuda


In [ ]:
import torch.optim as optim
import torch.nn as nn

#calculate how close model prediction is to actual classification
criterion = nn.CrossEntropyLoss()
#optimizes based on gradients computed from loss on last layer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

cuda


In [ ]:
epochs = 10
#10 iterations
#tried with 2 epochs -> 95% accuracy
for epoch in range(epochs):
    # training mode - actually changes parameters
    model.train()
    runningLoss1, runningCorrects = 0.0, 0

    #iterates through the random batches we defined in trainLoader above
    for inputs, labels in trainLoader:
        #put data on same device (gpu)
        inputs, labels = inputs.to(device), labels.to(device)

        #clear gradients, forward pass
        optimizer.zero_grad()
        outputs = model(inputs)

        #compute loss, backward prop.
        loss = criterion(outputs, labels)
        loss.backward()

        #applies this to model's paramaters
        optimizer.step()

        # from probability distribution, picks the class w greatest probability
        preds = outputs.argmax(dim=1)
        runningLoss1 += loss.item() * inputs.size(0)
        runningCorrects += (preds == labels).sum().item()

    #avg loss per image
    trainLoss = runningLoss1 / len(trainDS)
    trainAcc = runningCorrects / len(trainDS)

    # validating mode - tests how well model does with current training
    model.eval()
    runningLoss2, valCorrects = 0.0, 0
    with torch.no_grad():
        for inputs, labels in valLoader:
            #put data on same device (gpu)
            inputs, labels = inputs.to(device), labels.to(device)
            #forward pass
            outputs = model(inputs)
            #compute loss (no back-propagation bc we aren't training model here)
            loss = criterion(outputs, labels)
            # from probability distribution, picks the class w greatest probability
            preds = outputs.argmax(dim=1)
            runningLoss2 += loss.item() * inputs.size(0)
            valCorrects += (preds == labels).sum().item()
    #avg loss per image
    valLoss = runningLoss2 / len(valDS)
    valAcc = valCorrects / len(valDS)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train - Loss: {trainLoss:.4f}, Accuracy: {trainAcc:.4f} | "
          f"Val - Loss: {valLoss:.4f}, Accuracy {valAcc:.4f}")


Epoch 1/10 | Train - Loss: 0.1518, Accuracy: 0.9519 | Val - Loss: 0.1344, Accuracy 0.9569
Epoch 2/10 | Train - Loss: 0.1340, Accuracy: 0.9558 | Val - Loss: 0.1176, Accuracy 0.9606
Epoch 3/10 | Train - Loss: 0.1243, Accuracy: 0.9584 | Val - Loss: 0.1232, Accuracy 0.9585
Epoch 4/10 | Train - Loss: 0.1149, Accuracy: 0.9614 | Val - Loss: 0.1115, Accuracy 0.9634
Epoch 5/10 | Train - Loss: 0.1115, Accuracy: 0.9629 | Val - Loss: 0.1089, Accuracy 0.9637
Epoch 6/10 | Train - Loss: 0.1034, Accuracy: 0.9654 | Val - Loss: 0.1086, Accuracy 0.9634
Epoch 7/10 | Train - Loss: 0.1023, Accuracy: 0.9649 | Val - Loss: 0.1064, Accuracy 0.9648
Epoch 8/10 | Train - Loss: 0.0989, Accuracy: 0.9661 | Val - Loss: 0.1205, Accuracy 0.9606
Epoch 9/10 | Train - Loss: 0.0937, Accuracy: 0.9674 | Val - Loss: 0.1156, Accuracy 0.9606
Epoch 10/10 | Train - Loss: 0.0933, Accuracy: 0.9669 | Val - Loss: 0.1010, Accuracy 0.9655


In [ ]:
# Epoch 10/10 | Train - Loss: 0.0933, Accuracy: 0.9669 | Val - Loss: 0.1010, Accuracy 0.9655, Time: 54m

In [ ]:
torch.save(model.state_dict(), "plant_detect_resnet18_state.pth")

In [ ]:
from google.colab import files
files.download("plant_detect_resnet18_state.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for name, param in model.named_parameters():
    print(name, param.mean().item())

conv1.weight 2.941970706160646e-05
bn1.weight 0.25757724046707153
bn1.bias 0.18112018704414368
layer1.0.conv1.weight -0.0030870495829731226
layer1.0.bn1.weight 0.3396005630493164
layer1.0.bn1.bias -0.0341365709900856
layer1.0.conv2.weight -0.0008893321501091123
layer1.0.bn2.weight 0.33305495977401733
layer1.0.bn2.bias 0.00346287339925766
layer1.1.conv1.weight -0.00242004101164639
layer1.1.bn1.weight 0.32869213819503784
layer1.1.bn1.bias -0.08357393741607666
layer1.1.conv2.weight -0.0012602501083165407
layer1.1.bn2.weight 0.3924297094345093
layer1.1.bn2.bias -0.029983950778841972
layer2.0.conv1.weight -0.0014537476236000657
layer2.0.bn1.weight 0.31641942262649536
layer2.0.bn1.bias -0.06734631955623627
layer2.0.conv2.weight -0.0012476658448576927
layer2.0.bn2.weight 0.3275725841522217
layer2.0.bn2.bias -0.003555297153070569
layer2.0.downsample.0.weight -0.002587598981335759
layer2.0.downsample.1.weight 0.19508540630340576
layer2.0.downsample.1.bias -0.003555297153070569
layer2.1.conv1.we